# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [323]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
from joblib import dump, load
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score, average_precision_score

from sklearn.model_selection  import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cerion\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [253]:
dataBasePath = 'sqlite:///Data/DisasterResponse.db'
databaseTableName = 'messagesDataTable'

In [254]:
# load data from database
engine = create_engine(dataBasePath)
df = pd.read_sql(databaseTableName, engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [255]:
#X = df[['message','original', 'genre']]
X = df['message']
Y = df.drop(columns=['id', 'message','original', 'genre'])

In [256]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [257]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [258]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(x).lower().strip() for x in tokens]
    return clean_tokens

In [260]:
ost = tokenize('   Hello aLl happy worlds')
ost

['hello', 'all', 'happy', 'world']

In [131]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
import re
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [261]:
pipeline = Pipeline([    
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),       
            ('clf', MultiOutputClassifier(RandomForestClassifier()))])
 


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [272]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [273]:
Y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7917,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25322,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22191,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
18442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22826,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17975,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14608,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [274]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [275]:
Y_pred = pipeline.predict(X_test)

In [315]:
#Evaluate the model and print the accuracy

def EvaluateModel(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    
    total_hits = np.sum(np.sum(Y_pred == Y_test))
    total_misses = np.sum(np.sum(Y_pred != Y_test))
    total_accuracy = total_hits/(total_hits + total_misses)
       
    target_names = [name for name in Y.columns]

    precisions = []
    recalls = []
    f1scores = []
    for (name, col) in zip(target_names, range(len(target_names))):
        y_test = Y_test[name].values
        y_pred = Y_pred[:, col]
        
        if(np.max(y_test) <= 1):
            #Only one category
            precisions.append(precision_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred))
            f1scores.append(f1_score(y_test, y_pred))
        print(f'Category: {name}')
        print(classification_report(y_test, y_pred))
        print('-'*42)
     
    
    print(f'Total Accuracy: {total_accuracy}')
    print(f'Average Precission: {np.average(precisions)}')
    print(f'Average Recall: {np.average(recalls)}')
    print(f'Average F1 Score: {np.average(f1scores)}')
    

In [316]:
EvaluateModel(pipeline, X_test, Y_test)

Category: related
              precision    recall  f1-score   support

           0       0.74      0.27      0.40      1266
           1       0.80      0.97      0.88      3938
           2       0.70      0.17      0.28        40

    accuracy                           0.79      5244
   macro avg       0.75      0.47      0.52      5244
weighted avg       0.78      0.79      0.76      5244

------------------------------------------
Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.88      0.40      0.55       895

    accuracy                           0.89      5244
   macro avg       0.89      0.70      0.74      5244
weighted avg       0.89      0.89      0.87      5244

------------------------------------------
Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00  

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4833
           1       0.93      0.40      0.56       411

    accuracy                           0.95      5244
   macro avg       0.94      0.70      0.76      5244
weighted avg       0.95      0.95      0.94      5244

------------------------------------------
Category: storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4758
           1       0.77      0.46      0.58       486

    accuracy                           0.94      5244
   macro avg       0.86      0.72      0.77      5244
weighted avg       0.93      0.94      0.93      5244

------------------------------------------
Category: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5191
           1       0.00      0.00      0.00        53

    accuracy                           0.99      5244
   macro av

### 6. Improve your model
Use grid search to find better parameters. 

In [317]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000028BDE084160>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [ ]:
parameters = {
    #'tfidf__norm':['l1', 'l2'],
    #'clf__estimator__n_estimators':[10,100,1000,5000],
    'clf__estimator__n_estimators':[100,2000],
    'clf__estimator__max_depth':[2, 5, 10],
    'clf__estimator__min_samples_leaf': [1,10],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, Y_train)

In [ ]:
cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [319]:
EvaluateModel(cv, X_test, Y_test)

Category: related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1266
           1       0.75      1.00      0.86      3938
           2       0.00      0.00      0.00        40

    accuracy                           0.75      5244
   macro avg       0.25      0.33      0.29      5244
weighted avg       0.56      0.75      0.64      5244

------------------------------------------
Category: request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4349
           1       0.00      0.00      0.00       895

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.69      0.83      0.75      5244

------------------------------------------
Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00  

C:\Users\cerion\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cerion\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cerion\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cerion\AppData\Roaming\Python\Python38\

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

------------------------------------------
Category: water
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4905
           1       0.00      0.00      0.00       339

    accuracy                           0.94      5244
   macro avg       0.47      0.50      0.48      5244
weighted avg       0.87      0.94      0.90      5244

------------------------------------------
Category: food
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      4649
           1       0.00      0.00      0.00       595

    accuracy                           0.89      5244
   macro avg       0.44      0.50      0.47      5244
weighted av

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [322]:
dump(cv, 'model.pkl') 

TypeError: dumps() missing required argument 'obj' (pos 1)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.